# OpenStreetMap Data Project


*created by: Tim Urian* 
<tim.urian@gmail.com>


In this project I downloaded 5+ GB of [OpenStreetMap](https://www.openstreetmap.org) data from around the [Los Angeles Metro Area](https://mapzen.com/data/metro-extracts/metro/los-angeles_california/).  

I currently live in the Los Angeles and have gotten to meet some of the people who update and maintain OSM data at tech meetups around the city. I was excited to play with a much bigger dataset than I’ve dealt with before, and to get more experience with SQL. 

In the first section of this project I clean the data and create csv's to be imported into an SQLite database using python. The data is parsed iteratively to deal with the large filesize.

The issues in the data I addressed included:
* __Streetnames__
 * Poorly Capitalized 
 * Overly Abbreviated 
 
 
* __Zip Codes__
 * Standardized to 5 digits
 * Contained extra spaces, characters
 * Eliminated anything that wasn't a zipcode
 * Eliminated zips not in California
 * Some began with 'CA'

Some of the problems were issues I expected (the +4 zip codes for example), others were found in a circular process of normalizing data with the most glaring problems, then seeing what issues I had missed and whether or not they could be solved programmatically. 

In the second section of this document I use SQL to explore the dataset and answer some questions that interest me. 

Cleaning and Processing the Data
======

In [1]:
# Import Statements
##################

import xml.etree.cElementTree as ET
import csv
import re
import sys
import codecs
import string
import logging

In [2]:
# Global Constants
##################

NODE_COLUMNS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
TAG_COLUMNS = ['id', 'key', 'value', 'type']
WAY_COLUMNS = ['id', 'user', 'uid', 'version', 'timestamp', 'changeset']
WAY_NODES_COLUMNS = ['id', 'node_id', 'position']

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

SCHEMA_DICT = {'node': NODE_COLUMNS, 
               'tag': TAG_COLUMNS,
               'way': WAY_COLUMNS,
               'nd': WAY_NODES_COLUMNS}

filename ='los-angeles_california.osm'
logging.basicConfig(filename='parsing.log', level=logging.INFO)

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


In [3]:
# Helper methods
################

def get_element(osm_file, tags=('node', 'way')):
    """Yield Element if it is the right type of tag"""
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, element in context:
        if event == 'end' and element.tag in tags:
            yield element
            root.clear()

def format_tag_key(tag, key='key'):
    """Returns seperate, corrected key and type values"""
    if PROBLEMCHARS.search(tag[key]) is None:
        if bool(LOWER_COLON.search(tag[key])):
            split_tag = tag[key].split(':', 1)
            tag[key] = split_tag[1]
            tag['type'] = split_tag[0]
        else:
            tag['type'] = 'regular'
    else:
        logging.info('Tag has problem character: %s skipping.', repr(tag[key]))
        tag['type'] = 'skip'
    return tag

def new_shape_element(element):
    """Returns dict correctly shaped for writing to csv file"""
    row = {}
    tags = []
    nds= []
    
    for attribute in SCHEMA_DICT[element.tag]:
        if attribute in element.attrib.keys():
            row[attribute] = element.attrib[attribute]
    shaped_dict = {element.tag : row}
    
    for tag in element.iter('tag'):
        tag_as_dict = {'id':row['id'], 'key':tag.attrib['k'], 'value':tag.attrib['v'], 'type':'regular'}
        tag_as_dict = format_tag_key(tag_as_dict)
        if tag_as_dict['type'] != 'skip':
            tags.append(tag_as_dict)
    if tags:
        shaped_dict['tags'] = tags
    
    for position, nd in enumerate(element.iter('nd')):
        nd_as_dict = {'id':row['id'], 'node_id':nd.attrib['ref'], 'position':position}
        nds.append(nd_as_dict)
    if nds:
        shaped_dict['way_nodes'] = nds
    
    return shaped_dict


def add_row_to_csv(filename, row, fieldnames):
    with open(filename, 'wb') as f:
        writer = csv.DictWriter(sys.stderr, fieldnames=fieldnames)
        writer.writerow(row)

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


In [4]:
# Data Auditing Functions 
##########################

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
is_postcode_re = re.compile('^\d{5}$')
is_postcode_plus_four_re = re.compile(r'.*(\d{5}(\-\d{4}))$')

expected_streetnames = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Highway", "Pike", "Way", "Circle", "Terrace", "Alley","Canal","Center","Circle",
                       "Cove", "Trail", "Way" ]
streetname_mapping = {'Al':'Alley',
                     'Ave':'Avenue',
                     'Av':'Avenue',
                     'Blvd':'Boulevard',
                     'Bv':'Boulevard',
                      'Boulvard':'Boulevard',
                      'Ca':'Canal',
                      'Cn':'Center',
                      'Cr':'Cirle',
                      'Cir':'Circle',
                      'Ct':'Court',
                      'Cv':'Cove',
                      'Dr': 'Drive',
                      'Hwy':'Highway',
                     'Pkwy': 'Parkway',
                      'Pky':'Parkway',
                      'Pl':'Place',
                     'Rd':'Road',
                     'St':'Street',
                      'Sq':'Square',
                      'Tr':'Trail',
                      'Trl':'Trail',
                      'Ln': 'Lane',
                     'Wy':'Way',
                     }

def correct_streetname(streetname):
    """Returns a corrected streetname"""
    
    #Capitalize all words in streetname string, remove any '.'s 
    streetname_capitalized = string.capwords(streetname).replace('.', '')

    #Grab just the type of street (ie Avenue, or Ave)
    street_type = street_type_re.search(streetname_capitalized)
    
    #Provided there is a street type, correct if not formatted as expected. 
    if street_type is not None:
        street_type = street_type.group()
        if street_type not in expected_streetnames:
            if street_type in streetname_mapping:
                new_street_type = streetname_mapping[street_type]
                streetname = streetname_capitalized.replace(street_type, new_street_type)
            else:
                streetname = streetname_capitalized
                logging.info("No streetname mapping correction: %s", streetname)
        else:
            streetname = streetname_capitalized
    else:
        logging.info('street_type is NONE with this streetname: %s', streetname)
    #TODO Add support for Spanish Street Prefixes
    return streetname

   
def correct_zipcode(zipcode):
    """Returns a corrected zipcode"""
    if is_postcode_re.search(zipcode) is None:
        if is_postcode_plus_four_re.match(zipcode) is not None:
            zipcode =  zipcode[:5]
        elif zipcode[:3].upper() == 'CA ':
            zipcode = zipcode[3:]
            if is_postcode_re.search(zipcode) is not None:
                return zipcode
            else:
                zipcode = ''
        elif is_postcode_re.search(zipcode.replace(' ', '')) is not None:
            return zipcode.replace(' ', '')
        else:
            zipcode = ''
    return zipcode


def is_zip_in_california(zipcode):
    """Checks if the zipcode could be in California"""
    #First three digits of zip should be between 900 and 961 
    if zipcode is not '':
        try:
            if 900 <= int(zipcode[:3]) <= 961:
                return True
            else:
                logging.info('Non CA zip code found: %s', zipcode)
                return False
        except:
            logging.info("Can't check for Califonia: ", zipcode)

In [ ]:
# Main
#######

with codecs.open(NODES_PATH, 'w') as nodes_file, \
    codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
    codecs.open(WAYS_PATH, 'w') as ways_file, \
    codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
    codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

    nodes_writer = UnicodeDictWriter(nodes_file, NODE_COLUMNS)
    node_tags_writer = UnicodeDictWriter(nodes_tags_file, TAG_COLUMNS)
    ways_writer = UnicodeDictWriter(ways_file, WAY_COLUMNS)
    way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_COLUMNS)
    way_tags_writer = UnicodeDictWriter(way_tags_file, TAG_COLUMNS)

    nodes_writer.writeheader()
    node_tags_writer.writeheader()
    ways_writer.writeheader()
    way_nodes_writer.writeheader()
    way_tags_writer.writeheader()

    for element in get_element(filename):
        element_to_write = new_shape_element(element)
        if 'tags' in element_to_write:
            for tag in element_to_write['tags']:
                if tag['key'] == 'street':
                    tag['value'] = correct_streetname(tag['value'])
                if tag['key'] == 'postcode':
                    new_zip = correct_zipcode(tag['value'])
                    if is_zip_in_california(new_zip):
                        tag['value'] = new_zip
                    else: 
                        tag['value'] = ''

        if 'node' in element_to_write:
            nodes_writer.writerow(element_to_write['node'])
            if 'tags' in element_to_write:
                node_tags_writer.writerows(element_to_write['tags'])
        if 'way' in element_to_write:
            ways_writer.writerow(element_to_write['way'])
            if 'way_nodes' in element_to_write:
                way_nodes_writer.writerows(element_to_write['way_nodes'])
            if 'tags' in element_to_write:
                way_tags_writer.writerows(element_to_write['tags']) 

logging.info('Finished')

The streetname mapping was created iteratively with the help of LA County's helpful [Street Name Policy](https://dpw.lacounty.gov/services/roads/streetname/policy.pdf). Much of the time I would sample a subset of the data and see what didn't conform to my expectations to add new rules for correcting the data.

SQL Exploration
================

First, let's find out some basic info about the data.

__Number of users who contributed nodes:__

    SELECT COUNT(DISTINCT user) FROM nodes;
*342*


__Number of Nodes, Ways, and Node Tags:__

    SELECT COUNT(id) FROM nodes;
*24,186,798*
    
    SELECT COUNT(id) FROM ways;
*2,443,996*
    
    SELECT COUNT(*) FROM nodes_tags;
*938160*

__Number of nodes with tag information:__
    
    SELECT COUNT(DISTINCT key) FROM nodes_tags;
*817*

__Top 10 keys (labels for the value column) from nodes_tags:__

    SELECT COUNT(key), key
    FROM nodes_tags
    GROUP BY key
    ORDER BY COUNT(key) DESC LIMIT 10;
|Number| Key|
|----|-----|
|89456|highway|
|86950|source|
|76347|street|
|76264|housenumber|
|72844|city|
|72415|postcode|
|40172|power|
|40154|building|
|38550|natural|
|34357|country|

I also took a look at the top 20 and found a key called 'amenity' which I will explore later.

The key 'natural' looks interesting, let's see what natural features OSM users are adding. 

    SELECT COUNT(value), value
    FROM nodes_tags
    WHERE key LIKE ‘natural’
    GROUP BY value
    ORDER BY COUNT(value) DESC LIMIT 10;

|COUNT(value) | value|
|-----|-----|
|37572|tree|
|568|peak|
|108|bay|
|93|spring|
|54|cave_entrance|
|34|beach|
|19|bush|
|19|succulent plant|
|10|stone|
|9|saddle|

Well, the LA area certainly has more cave entrances than I expected! Very cool. However, the number of beaches seems low, I would like to see how they're being labeled. 

Let's explore the 'Amenities' Key in the same way. 

    SELECT COUNT(value), value FROM nodes_tags 
    WHERE key = ‘amenity’ 
    GROUP BY value
    ORDER BY COUNT(value) DESC LIMIT 10;


|COUNT(value) | value|
|-----|-----|
|3697|place_of_worship|
|2877|school|
|2620|restaurant|
|1422|fast_food|
|823|fuel|
|787|bench|
|758|cafe|
|694|fountain|
|589|toilets|
|520|drinking_water|

Being a coffee lover, the 758(!!!) cafes jump out at me. 

Let's get a little more complicated and see what zip codes have the most cafes. I'm betting on Silverlake or Los Feliz!

    select count(nodes_tags.value), i.value
    from nodes_tags
    JOIN (select distinct(id), value, key FROM nodes_tags WHERE key='postcode') i
    ON nodes_tags.id=i.id
    WHERE nodes_tags.value='cafe'
    GROUP BY i.value
    ORDER BY count(nodes_tags.value) DESC
    LIMIT 5;

|Number | Zip |
|-----|-----|
|7|90024|
|5|90027|
|5|92672|
|4|90029|
|4|90291|

Seems like my intuition was only partially correct. Westwood actually came in #1 with 7 cafes, followed by Los Feliz and San Clemente - which also gives you an idea for how far south the dataset extends. 

It should be noted that zip codes aren't a great measure for cafe density because they are arbitrarily drawn and have sizes that vary. For a more thorough exploration of cafes in LA I would create a heatmap using the latitude and longitudes, make certain that there weren't cafes listed as restaurants in this dataset, and scrape new data from aggregators like Yelp. 

Finally, let's look at the top five sources for the node's and way's tags.

    SELECT COUNT(*), value 
    FROM (SELECT * FROM nodes_tags UNION ALL 
          SELECT * FROM ways_tags)
    WHERE key = 'source'
    GROUP BY value
    ORDER BY 1 DESC
    LIMIT 5;

|COUNT | value |
|-----|-----|
|108353|tiger_import_dch_v0.6_20070809|
|51774|bing_imagery|
|15751|"SanGIS Addresses Public Domain (http://www.sangis.org/)"|
|11738|Yahoo|
|10867|Bing|

This makes sense with some of the things we saw while cleaning, like the many datapoints coming from the Tiger systems.

### Ideas for Additional Improvement

After becoming so familiar with this data I have begun to wonder if there isn't a way to make sure the data is normalized going into the OSM system. Perhaps there are technical barriers to this, but ensuring that certain values are entered uniformly (5 digit zips being a simple example) would make large scale analyses much easier. 

One potential issue with this idea is making a service that is largely kept update by volunteers harder to use, therefore makeing contributors less likely to loan their time. 

### Summary

Dealing with a large (for me) dataset where much of the information was typed by humans was illuminating - I now know why there are so many articles that mention how much of a data professional's time is spent *cleaning* data. 

It was great to parse xml iteratively in python (instead of reading the whole document into memory), and exploring with SQL was an excellent experience - very different from what I've done with pandas in the past. I've now done many, many SQL tutorials and it's left me wanting  more - the chance to answer real business questions perhaps, or to explore how to go back and forth between SQL and visualization tools. I feel that my firm grip on relational databases will be important as the data I deal with grows more and more complex.

And, in the meantime, I've found a couple hundred fixes I can contribute to the OSM project!